### Sequence numbering
In this notebook, we illustrate how to use the SingleChainAnnotator tool to determine
whether a sequence is heavy or light (lambda, kappa) chain and number it. We also demonstrate
how to use AntPack to extract specific CDRs or framework regions and how to merge numbering
for antibody sequences of different lengths so that you can write your sequences to a csv file
as an MSA or encode them as a fixed-length array for ML.

In [1]:
from antpack import SingleChainAnnotator

my_sequence = "AAAAAAAEVHLQQSGAELMKPGASVKISCKASGYTFITYWIEWVKQRPGHGLEWIGDILPGSGSTNYNENFKGKATFTADSSSNTAYMQLSSLTSEDSAVYYCARSGYYGNSGFAYWGQGTLVTVSA"

additional_sequences = ["EVQLVQSGAEVKKPGESLKISCKGSGYSFTSYWIGWVRQMPGKGLEWMGIIYPGDSDTRYSPSFQGQVTISADKSISTAYLQWSSLKASDTAMYYCARPHYYDSLDAFDIWGQGTMVTVSS",
                        "ITLKESGPTLVKPTQTLTLTCTFSGFSLSTSGMGVSWIRQPPGKALEWLAHIYWDDDKRYNPSLKSRLTITKDTSKNQVVLTMTNMDPVDTATYYCARLYGFTYGFAYWGQGTLVTVSS",
                        "EVKLQESGPGKLQPSQTLSLTCSFSGFSLTTSGIGVGWIRQPSGKGLEWLAHIWWSASKYYNTALKSRLTISKDTSNNQVFLKIASVDTADTATYYCARAYYGNYGGYYFDYWGQGTTLTVSS"]

By passing ["H", "K", "L"] (the default) we ensure the annotator will align each sequence to
heavy, kappa and lambda (kappa and lambda are different variants of light chains)
and will determine the type of the chain based on which option returns the best
alignment. If we KNOW our chains are all "H", we could pass "H" as the only
option, and this will improve speed slightly. In general however this isn't
necessary.

In [2]:
my_annotator = SingleChainAnnotator(["H", "K", "L"], scheme = "imgt")

In [3]:
numbering, percent_identity, chain_type, err_message = my_annotator.analyze_seq(my_sequence)

The numbering is a list of the same length as the input sequence where each element is either
"-" (a gap, meaning there's no numbering assignment that corresponds to that amino acid or
a numbering assignment.

In [4]:
print(f"{numbering}\n\n")
print(f"{my_sequence}\n\n")
print([(a, z) for a, z in zip(numbering, my_sequence)])

['-', '-', '-', '-', '-', '-', '-', '1', '2', '3', '4', '5', '6', '7', '8', '9', '11', '12', '13', '14', '15', '16', '17', '18', '19', '20', '21', '22', '23', '24', '25', '26', '27', '28', '29', '30', '35', '36', '37', '38', '39', '40', '41', '42', '43', '44', '45', '46', '47', '48', '49', '50', '51', '52', '53', '54', '55', '56', '57', '58', '59', '62', '63', '64', '65', '66', '67', '68', '69', '70', '71', '72', '74', '75', '76', '77', '78', '79', '80', '81', '82', '83', '84', '85', '86', '87', '88', '89', '90', '91', '92', '93', '94', '95', '96', '97', '98', '99', '100', '101', '102', '103', '104', '105', '106', '107', '108', '109', '110', '111', '112', '113', '114', '115', '116', '117', '118', '119', '120', '121', '122', '123', '124', '125', '126', '127', '128']


AAAAAAAEVHLQQSGAELMKPGASVKISCKASGYTFITYWIEWVKQRPGHGLEWIGDILPGSGSTNYNENFKGKATFTADSSSNTAYMQLSSLTSEDSAVYYCARSGYYGNSGFAYWGQGTLVTVSA


[('-', 'A'), ('-', 'A'), ('-', 'A'), ('-', 'A'), ('-', 'A'), ('-', 'A'), ('-', 'A'), ('1', '

SingleChainAnnotator determined this was a heavy chain ("H"). "K" and "L" both correspond to light
chains (kappa and lambda).

In [5]:
print(chain_type)

H


A low percent identity (<< 0.8) could mean this isn't an antibody sequence, contains large
deletions, or some other issue. In this case, no problems.

In [6]:
print(percent_identity)

0.9659090909090909


Finally, if the error message is anything other than "", something went wrong. The most common
error message indicates an unexpected amino acid was found at a highly conserved position --
this usually occurs if there is a very large N or C terminal deletion. In this case we're fine.

In [7]:
print(err_message)

Note that above we used ``analyze_seq`` to analyze a single sequence. If we have a list
of sequences we can use ``analyze_seqs``, or we can loop over our list and feed each
sequence to ``analyze_seq`` as we go.

Once we have the numbering, we can use it to extract CDRs using the CDR definitions for
an appropriate numbering scheme. Alternatively, we can ask AntPack to tell us what
the CDR and framework regions are for that scheme at the same time that we number the
sequence. To do so, we just set ``get_region_labels=True`` when calling ``analyze_seq``.
``analyze_seq`` will now return an additional list which indicates for each numbered
position whether it is "-", "fmwk1", "cdr1", "fmwk2", "cdr2", "fmwk3", "cdr3", or "fmwk4".
The annotator uses the CDR definitions corresponding to the numbering scheme you selected
when you created it. Here's what this looks like:

In [8]:
numbering, percent_identity, chain_type, err_message, region_labels = \
        my_annotator.analyze_seq(my_sequence, get_region_labels = True)
print(region_labels)

['-', '-', '-', '-', '-', '-', '-', 'fmwk1', 'fmwk1', 'fmwk1', 'fmwk1', 'fmwk1', 'fmwk1', 'fmwk1', 'fmwk1', 'fmwk1', 'fmwk1', 'fmwk1', 'fmwk1', 'fmwk1', 'fmwk1', 'fmwk1', 'fmwk1', 'fmwk1', 'fmwk1', 'fmwk1', 'fmwk1', 'fmwk1', 'fmwk1', 'fmwk1', 'fmwk1', 'fmwk1', 'cdr1', 'cdr1', 'cdr1', 'cdr1', 'cdr1', 'cdr1', 'cdr1', 'cdr1', 'fmwk2', 'fmwk2', 'fmwk2', 'fmwk2', 'fmwk2', 'fmwk2', 'fmwk2', 'fmwk2', 'fmwk2', 'fmwk2', 'fmwk2', 'fmwk2', 'fmwk2', 'fmwk2', 'fmwk2', 'fmwk2', 'fmwk2', 'cdr2', 'cdr2', 'cdr2', 'cdr2', 'cdr2', 'cdr2', 'cdr2', 'cdr2', 'fmwk3', 'fmwk3', 'fmwk3', 'fmwk3', 'fmwk3', 'fmwk3', 'fmwk3', 'fmwk3', 'fmwk3', 'fmwk3', 'fmwk3', 'fmwk3', 'fmwk3', 'fmwk3', 'fmwk3', 'fmwk3', 'fmwk3', 'fmwk3', 'fmwk3', 'fmwk3', 'fmwk3', 'fmwk3', 'fmwk3', 'fmwk3', 'fmwk3', 'fmwk3', 'fmwk3', 'fmwk3', 'fmwk3', 'fmwk3', 'fmwk3', 'fmwk3', 'fmwk3', 'fmwk3', 'fmwk3', 'fmwk3', 'fmwk3', 'fmwk3', 'cdr3', 'cdr3', 'cdr3', 'cdr3', 'cdr3', 'cdr3', 'cdr3', 'cdr3', 'cdr3', 'cdr3', 'cdr3', 'cdr3', 'cdr3', 'fmwk4', 'fm

Notice that the list ``region_labels`` is the same length as ``my_sequence`` or as ``numbering``. This provides
an easy way to extract a specific cdr or framework if desired.

What if we number a list of sequences, some have more positions than others, and now we
want to generate a list that contains all of the positions observed in one or more sequences
in the correct order? This is useful if for example you want to write all the sequences to
say a csv file where each row is the same length, or convert all your sequences to a fixed-length
array for a machine learning task (e.g. as input to a gradient boosted trees algorithm).

AntPack makes it easy to do this. First, we'll number the sequences, then we'll create a set containing
all the positions observed in one or more sequences. AntPack can sort that set so the
positions are in the correct order for the numbering scheme we're using. Then we can easily
convert that to a dict and use it to process our sequences as illustrated below. The
sequences we're using here are all heavy chain; if we had both heavy and light, of course,
we might want to handle them separately.

It's important to note that when sorting position codes, "-" is *not* a valid position code,
so we have to remove it before sorting, otherwise we'll get an error.

In [9]:
all_observed_position_codes = set()

numbering = []

for seq in additional_sequences:
    results = my_annotator.analyze_seq(seq)
    numbering.append(results[0])
    if results[3] != "":
        # If there is an error message indicating a bad alignment, do something here
        pass

    # Add all position codes to the set
    all_observed_position_codes.update(results[0])

all_observed_position_codes = list(all_observed_position_codes)

#It's important to note that when sorting position codes, "-" is *not* a valid position code,
#so we have to remove it before sorting, otherwise we'll get an error.
all_observed_position_codes = [a for a in all_observed_position_codes if a != "-"]

print(all_observed_position_codes)

['63', '11', '76', '80', '92', '93', '4', '112A', '69', '86', '37', '23', '40', '72', '31', '123', '64', '54', '48', '107', '13', '124', '17', '2', '65', '75', '89', '97', '52', '55', '41', '6', '102', '71', '101', '7', '18', '115', '82', '22', '121', '21', '94', '106', '67', '58', '79', '110', '14', '35', '105', '88', '27', '56', '43', '103', '91', '90', '53', '26', '66', '20', '104', '38', '84', '9', '50', '16', '44', '95', '126', '114', '19', '111', '78', '12', '117', '122', '70', '77', '36', '24', '100', '119', '128', '1', '49', '113', '46', '42', '116', '45', '62', '3', '125', '127', '85', '59', '112', '51', '87', '108', '5', '30', '96', '39', '68', '47', '8', '15', '83', '28', '29', '111A', '81', '109', '34', '25', '99', '57', '120', '118', '98', '74']


Notice that the list of observed position codes isn't in order. We can fix that with a single call. It's important
to supply the correct scheme when you call ``sort_position_codes`` -- different schemes have some "quirks" in
terms of how positions are ordered, and if you send Martin numbering to this function for example even
though your numbering was actually IMGT, you might get unexpected results...

In [10]:
all_observed_position_codes = my_annotator.sort_position_codes(all_observed_position_codes, scheme = "imgt")
print(all_observed_position_codes)

['1', '2', '3', '4', '5', '6', '7', '8', '9', '11', '12', '13', '14', '15', '16', '17', '18', '19', '20', '21', '22', '23', '24', '25', '26', '27', '28', '29', '30', '31', '34', '35', '36', '37', '38', '39', '40', '41', '42', '43', '44', '45', '46', '47', '48', '49', '50', '51', '52', '53', '54', '55', '56', '57', '58', '59', '62', '63', '64', '65', '66', '67', '68', '69', '70', '71', '72', '74', '75', '76', '77', '78', '79', '80', '81', '82', '83', '84', '85', '86', '87', '88', '89', '90', '91', '92', '93', '94', '95', '96', '97', '98', '99', '100', '101', '102', '103', '104', '105', '106', '107', '108', '109', '110', '111', '111A', '112A', '112', '113', '114', '115', '116', '117', '118', '119', '120', '121', '122', '123', '124', '125', '126', '127', '128']


And now all the position codes are in order. This makes it easy to turn our sequences into a multiple sequence
alignment in a file, or encode them as a fixed-length array as illustrated below:

In [11]:
position_dict = {k:i for i, k in enumerate(all_observed_position_codes)}

fixed_length_seqs = []

for seq, pos_codes in zip(additional_sequences, numbering):
    fixed_length_seq = ["-" for code in all_observed_position_codes]

    for letter, code in zip(seq, pos_codes):
        fixed_length_seq[position_dict[code]] = letter

    fixed_length_seqs.append("".join(fixed_length_seq))

print(fixed_length_seqs)

['EVQLVQSGAEVKKPGESLKISCKGSGYSF--TSYWIGWVRQMPGKGLEWMGIIYPGDSDTRYSPSFQGQVTISADKSISTAYLQWSSLKASDTAMYYCARPHYYD-SLDAFDIWGQGTMVTVSS', '-ITLKESGPTLVKPTQTLTLTCTFSGFSLSTSGMGVSWIRQPPGKALEWLAHIYWD-DDKRYNPSLKSRLTITKDTSKNQVVLTMTNMDPVDTATYYCARLYGF---TYGFAYWGQGTLVTVSS', 'EVKLQESGPGKLQPSQTLSLTCSFSGFSLTTSGIGVGWIRQPSGKGLEWLAHIWWS-ASKYYNTALKSRLTISKDTSNNQVFLKIASVDTADTATYYCARAYYGNYGGYYFDYWGQGTTLTVSS']
